In [1]:
using CausalELM
using CSV
using DataFrames
using Random

In [2]:
pension_df = CSV.read("pension.csv", DataFrame)
pension_df = pension_df[:, [10, 22, 13, 14, 15, 18, 20, 17, 24, 33]]
covariates, treatment, outcome = pension_df[:, 3:end], pension_df[:, 2], pension_df[:, 1]

(9915×8 DataFrame
  Row │ age    inc    fsize  marr   twoearn  db     pira   hown  
      │ Int64  Int64  Int64  Int64  Int64    Int64  Int64  Int64 
──────┼──────────────────────────────────────────────────────────
    1 │    31  28146      5      1        0      0      0      1
    2 │    52  32634      5      0        0      0      0      1
    3 │    50  52206      3      1        1      0      1      1
    4 │    28  45252      4      1        1      0      0      0
    5 │    42  33126      3      0        0      1      0      1
    6 │    49  76860      6      1        1      1      0      1
    7 │    40  57477      4      1        1      1      0      1
    8 │    58  14637      1      0        0      0      0      0
  ⋮   │   ⋮      ⋮      ⋮      ⋮       ⋮       ⋮      ⋮      ⋮
 9909 │    28  31926      2      1        1      0      0      0
 9910 │    49  64215      4      1        1      0      1      1
 9911 │    34  13500      1      0        0      1      0      0
 9912 

In [14]:
dr_learner = DoubleMachineLearning(covariates, treatment, outcome, num_feats=6)

DoubleMachineLearning([31.0 28146.0 … 0.0 1.0; 52.0 32634.0 … 0.0 1.0; … ; 41.0 56190.0 … 0.0 1.0; 28.0 26205.0 … 0.0 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0], [-3300.0, 61010.0, 8849.0, -6013.0, -2375.0, -11000.0, -16901.0, 1000.0, 0.0, 6400.0  …  -1436.0, 4500.0, 34739.0, -750.0, 40000.0, 172.0, 836.0, 6150.0, 14499.0, -5400.0], [31.0 28146.0 … 0.0 1.0; 52.0 32634.0 … 0.0 1.0; … ; 41.0 56190.0 … 0.0 1.0; 28.0 26205.0 … 0.0 0.0], "ATE", false, "regression", CausalELM.relu, 9915, 100, 6, 32, NaN, 5)

In [18]:
estimate_causal_effect!(dr_learner)

0.1134771453284956

In [ ]:
x, y = rand(10000, 7), rand(10000)

In [ ]:
learner = CausalELM.RegularizedExtremeLearner(x, y, 32, CausalELM.relu)

In [ ]:
CausalELM.fit!(learner)

In [ ]:
xs = [rand(1000, 8) for i in 1:100]
ys = [rand(1000) for i in 1:100]

learners = [CausalELM.ExtremeLearner(xs[i], ys[i], 5, CausalELM.relu) for i in 1:100]
CausalELM.fit!.(learners)

In [ ]:
mutable struct ELMEnsemble
    X::Array{Float64}
    Y::Array{Float64}
    elms::Array{CausalELM.ExtremeLearner}
end

function ELMEnsemble(
    X::Array{Float64}, 
    Y::Array{Float64}, 
    sample_size::Integer, 
    num_machines::Integer, 
    num_neurons::Integer
)
    rows = [rand(1:length(Y), length(Y)) for i in 1:num_machines]
    cols = [randperm(size(X, 2))[1:floor(Int64, sqrt(size(X, 2)))] for i ∈ 1:num_machines]
    xs, ys = [X[rows[i], cols[i]] for i ∈ eachindex(rows)], [Y[rows[i]] for i ∈ eachindex(rows)]
    elms = [CausalELM.ExtremeLearner(xs[i], ys[i], num_neurons, CausalELM.relu) for i ∈ 1:num_machines]

    return ELMEnsemble(X, Y, elms)
end

fit!(mod::ELMEnsemble) = CausalELM.fit!.(mod.elms)

In [ ]:
ensemble = ELMEnsemble(Matrix{Float64}(covariates), Float64.(outcome[:, 1]), 10000, 100, 10)

In [ ]:
fit!(ensemble)

In [ ]:
m1 = GComputation(x, rand(0:1, 10000), y, regularized=false)

In [ ]:
estimate_causal_effect!(m1)